In [1]:
import pandas as pd
tdate='2019-05-01'
# pivot_table 行转列

s=get_price(security=['601318.XSHG','000725.XSHE'],count=22,end_date=tdate,fields='money',panel=False)

s2=pd.pivot_table(s,index='code',columns=['time'],values=['money'])
s2.columns=s2.columns.droplevel()
weight=(s2.apply(lambda col:col/s2.apply(sum,axis=0)*100.0,axis=1)).apply(mean,axis=1)
weight


code
000725.XSHE    30.724088
601318.XSHG    69.275912
dtype: float64

In [16]:
import pandas as pd
from udf import to_date
import numpy as np


"""
分析沪深300百成分股，进而开发一个【沪深300指数增强策略】
1：获取沪深300的成分股的权重
2：计算沪深300的成分股的各行业的成交量占比、市值占比、营收占比、净利润占比
3：对上述的分析形成一个策略：
   1）超配沪深300成分里面最近连续3年收入占比增加，净利润占比也增加的行业，
   2）对于非连续增长的行业，只选取其行业龙头前几的股票
   形成投资组合，看是否能跑赢沪深300指数。

"""
# 重要参数，指数成分每年调整两次，不同日期，成分及其权重是不同的。
tdate='2019-05-01'


# 获取成分股及其权重 get_index_weights
components_df=get_index_weights('000300.XSHG',date=tdate)
# print(components_df.sort_values(['weight'],ascending=False).head())
hs300_components=list(components_df.index)

# 成分股成交量及按成交量计算的权重,最近22天成交量占比的平均
money=get_price(security=hs300_components,count=22,end_date=tdate,fields='money',panel=False)
money2=pd.pivot_table(money,index='code',columns=['time'],values=['money'])
money2.columns=money2.columns.droplevel()
weight_money=(money2.apply(lambda col:col/money2.apply(sum,axis=0)*100.0,axis=1)).apply(mean,axis=1)

weight_money.sort_values(ascending=False)
components_df['weight_money']=weight_money

# 成分股申万一、二级行业
sw_1=pd.Series({code:v['sw_l1']['industry_name']  for code,v in get_industry(security=hs300_components,date=tdate).items() if v.get('sw_l1','null')!='null'})
sw_2=pd.Series({code:v['sw_l2']['industry_name']      for code,v in get_industry(security=hs300_components,date=tdate).items() if v.get('sw_l2','null')!='null'})
    
components_df['sw_1']=sw_1
components_df['sw_2']=sw_2

# 成分股市值
df_cap = get_fundamentals(query(
          valuation.code,valuation.day,valuation.circulating_market_cap,valuation.market_cap
      ).filter(
          # 这里不能使用 in 操作, 要使用in_()函数
          valuation.code.in_(hs300_components)
      ), date=tdate)
df_cap=df_cap.set_index('code')
components_df['market_cap']=df_cap['market_cap']
components_df['c_market_cap']=df_cap['circulating_market_cap']

components_df.sort_values('weight',ascending=False).head(20)



,weight,display_name,date,weight_money,sw_1,sw_2,market_cap,c_market_cap
code,,,,,,,,
601318.XSHG,7.634,中国平安,2019-05-01,2.878814,非银金融I,保险II,15739.2881,9326.9238
600519.XSHG,4.006,贵州茅台,2019-05-01,1.917532,食品饮料I,饮料制造II,12235.3662,12235.3662
600036.XSHG,2.908,招商银行,2019-05-01,0.970277,银行I,银行II,8688.2363,7106.6714
000651.XSHE,2.189,格力电器,2019-05-01,2.185636,家用电器I,白色家电II,3343.5432,3318.5249
601166.XSHG,2.032,兴业银行,2019-05-01,0.992188,银行I,银行II,4138.2188,3795.2256
000333.XSHE,1.989,美的集团,2019-05-01,0.967210,家用电器I,白色家电II,3454.0776,3379.9280
000858.XSHE,1.626,五粮液,2019-05-01,1.922791,食品饮料I,饮料制造II,3973.2141,3885.3486
600887.XSHG,1.542,伊利股份,2019-05-01,0.845811,食品饮料I,食品加工II,1884.2196,1872.3362
600030.XSHG,1.484,中信证券,2019-05-01,2.191481,非银金融I,证券II,2791.7356,2261.2981


In [3]:
# 按行业分组按股票的指数权重排名
components_df['sw1_weight_rank']=components_df.groupby(['sw_1'])['weight'].rank(method='min',ascending=False)
components_df.sort_values('weight',ascending=False)
components_df

,weight,display_name,date,weight_money,sw_1,sw_2,market_cap,c_market_cap,sw1_weight_rank
code,,,,,,,,,
000001.XSHE,0.973,平安银行,2019-05-01,0.739823,银行I,银行II,2378.1021,2378.0791,8.0
000002.XSHE,1.149,万科A,2019-05-01,0.814182,房地产I,房地产开发II,3264.0588,2805.7412,1.0
000063.XSHE,0.626,中兴通讯,2019-05-01,2.172192,通信I,通信设备II,1347.9440,1103.8922,1.0
000069.XSHE,0.267,华侨城A,2019-05-01,0.278487,房地产I,房地产开发II,652.2245,557.8647,6.0
000100.XSHE,0.319,TCL集团,2019-05-01,0.364434,电子I,光学光电子II,486.4324,459.0813,4.0
000157.XSHE,0.182,中联重科,2019-05-01,0.166760,机械设备I,专用设备II,386.5455,311.2743,3.0
000402.XSHE,0.081,金融街,2019-05-01,0.068308,房地产I,房地产开发II,245.9889,245.8767,14.0
000408.XSHE,0.050,藏格控股,2019-05-01,0.049165,化工I,化学制品II,201.7705,45.1993,13.0
000413.XSHE,0.194,东旭光电,2019-05-01,0.566606,电子I,光学光电子II,353.5564,300.1184,8.0


In [4]:
from udf import get_fundamentals_ttm
# 计算成分股最近3个TTM的营收及归母利润
ttm,ttm_detail=get_fundamentals_ttm(tdate,3,hs300_components,income,['operating_revenue','np_parent_company_owners'])

ttm_report_df=pd.pivot_table(ttm,index=['code'],columns='ttm_lag',values=['operating_revenue','np_parent_company_owners'],aggfunc=np.sum)
# 列名重命名 revenue_1 表示 最近一期TTM收入，_2表示第2期TTM收入

ttm_report_df.columns=['profit_1','profit_2','profit_3','revenue_1','revenue_2','revenue_3']

ttm_report_df[(ttm_report_df.index=='601318.XSHG')]


['code', 'statDate', 'pubDate', 'operating_revenue', 'np_parent_company_owners']


,profit_1,profit_2,profit_3,revenue_1,revenue_2,revenue_3
code,,,,,,
601318.XSHG,1.272190e+11,9.173700e+10,6.474700e+10,1.044541e+12,9.356910e+11,7.692390e+11


In [5]:
# 合并成宽表
components_all=pd.concat([components_df,ttm_report_df],axis=1,sort=True)
components_all

,weight,display_name,date,weight_money,sw_1,sw_2,market_cap,c_market_cap,sw1_weight_rank,profit_1,profit_2,profit_3,revenue_1,revenue_2,revenue_3
000001.XSHE,0.973,平安银行,2019-05-01,0.739823,银行I,银行II,2378.1021,2378.0791,8.0,2.566900e+10,2.357000e+10,2.272700e+10,1.211660e+11,1.061000e+11,1.078950e+11
000002.XSHE,1.149,万科A,2019-05-01,0.814182,房地产I,房地产开发II,3264.0588,2805.7412,1.0,3.399840e+10,2.825128e+10,2.088478e+10,3.152283e+11,2.551335e+11,2.444551e+11
000063.XSHE,0.626,中兴通讯,2019-05-01,2.172192,通信I,通信设备II,1347.9440,1103.8922,1.0,-7.138210e+08,-2.052672e+09,-2.093321e+09,8.018862e+10,1.105970e+11,1.051193e+11
000069.XSHE,0.267,华侨城A,2019-05-01,0.278487,房地产I,房地产开发II,652.2245,557.8647,6.0,1.066670e+10,9.007155e+09,7.044544e+09,4.972368e+10,4.252431e+10,3.613615e+10
000100.XSHE,0.319,TCL集团,2019-05-01,0.364434,电子I,光学光电子II,486.4324,459.0813,4.0,3.516463e+09,2.947413e+09,1.789370e+09,1.173312e+11,1.118371e+11,1.084444e+11
000157.XSHE,0.182,中联重科,2019-05-01,0.166760,机械设备I,专用设备II,386.5455,311.2743,3.0,2.645205e+09,1.624119e+09,-1.888053e+08,3.135283e+10,2.435845e+10,2.227373e+10
000166.XSHE,0.284,申万宏源,2019-05-01,0.281725,非银金融I,证券II,1287.0532,1158.2689,12.0,5.029441e+09,4.620843e+09,5.334182e+09,1.785406e+10,1.342236e+10,1.460591e+10
000333.XSHE,1.989,美的集团,2019-05-01,0.967210,家用电器I,白色家电II,3454.0776,3379.9280,2.0,2.110357e+10,1.818733e+10,1.512969e+10,2.651331e+11,2.506939e+11,1.804583e+11
000338.XSHE,0.490,潍柴动力,2019-05-01,0.645895,汽车I,汽车零部件II,979.8334,524.1141,2.0,9.329244e+09,7.548043e+09,3.178559e+09,1.652554e+11,1.557931e+11,1.092925e+11
000402.XSHE,0.081,金融街,2019-05-01,0.068308,房地产I,房地产开发II,245.9889,245.8767,14.0,3.331530e+09,3.015951e+09,2.562817e+09,2.258178e+10,2.369543e+10,2.080230e+10


In [6]:
# 按申万一级行业汇总
# 计算各行业占比 指数权重、成交量占比、市值占比、股市数、收入占比，利润占比

t=components_all.groupby('sw_1').agg({
    'weight':['sum'],'weight_money':['sum'],'market_cap':['sum'],'date':['count']
    ,'revenue_1':['sum'],'profit_1':['sum']
    ,'revenue_2':['sum'],'profit_2':['sum']
    ,'revenue_3':['sum'],'profit_3':['sum']},sort=True)

# 
t.columns=['weight','weight_money', 'market_cap', 'stock_cnt','revenue_1','profit_1','revenue_2','profit_2','revenue_3','profit_3']

t_percent=t.div(t.sum())*100
# 计算各行业占比 指数权重、成交量占比、市值占比、股市数、收入占比，利润占比

t['market_cap_percent']=t_percent['market_cap']
t['revenue_1_p']=t_percent['revenue_1']
t['profit_1_p']=t_percent['profit_1']
t['revenue_2_p']=t_percent['revenue_2']
t['profit_2_p']=t_percent['profit_2']

t['revenue_3_p']=t_percent['revenue_3']
t['profit_3_p']=t_percent['profit_3']

t.sort_values(['weight'],ascending=False)


,weight,weight_money,market_cap,stock_cnt,revenue_1,profit_1,revenue_2,profit_2,revenue_3,profit_3,market_cap_percent,revenue_1_p,profit_1_p,revenue_2_p,profit_2_p,revenue_3_p,profit_3_p
sw_1,,,,,,,,,,,,,,,,,
非银金融I,17.793,17.124998,53217.2294,35,2.618876e+12,2.621986e+11,2.424829e+12,2.552985e+11,2.121902e+12,1.982183e+11,14.559756,8.785329,9.021832,9.380725,9.447035,9.386304,8.538036
银行I,17.160,8.301136,103208.9876,21,4.343419e+12,1.493945e+12,3.886534e+12,1.409825e+12,3.709606e+12,1.323915e+12,28.237051,14.570513,51.404246,15.035496,52.169017,16.409566,57.026186
食品饮料I,9.186,6.408974,25233.4475,9,3.279341e+11,8.617232e+10,2.783955e+11,6.818066e+10,2.250050e+11,4.751151e+10,6.903644,1.100094,2.965051,1.077004,2.522949,0.995317,2.046507
医药生物I,6.670,7.104281,17034.8707,30,5.939119e+11,4.835309e+10,4.794590e+11,4.387837e+10,4.016019e+11,3.597520e+10,4.660587,1.992347,1.663752,1.854841,1.623670,1.776500,1.549592
家用电器I,5.035,3.911575,9063.9698,6,6.879529e+11,5.959288e+10,6.051268e+11,5.344456e+10,4.543958e+11,3.994563e+10,2.479821,2.307819,2.050495,2.341001,1.977656,2.010035,1.720614
电子I,4.877,7.798681,13891.1435,19,9.325572e+11,5.307639e+10,4.455778e+11,4.166429e+10,3.592734e+11,2.934326e+10,3.800492,3.128373,1.826273,1.723768,1.541741,1.589258,1.263929
房地产I,4.608,3.919770,12499.5288,14,1.401989e+12,1.425579e+11,1.135941e+12,1.164412e+11,1.030400e+12,9.362219e+10,3.419759,4.703138,4.905186,4.394517,4.308775,4.558009,4.032674
交通运输I,3.290,2.897309,14650.8801,19,1.258959e+12,6.797630e+10,1.039592e+12,7.980135e+10,8.590861e+11,4.804023e+10,4.008349,4.223326,2.338955,4.021779,2.952960,3.800196,2.069281
建筑装饰I,3.250,3.314582,10807.4118,12,4.298882e+12,1.244122e+11,3.904085e+12,1.160240e+11,3.519898e+12,9.875676e+10,2.956811,14.421107,4.280824,15.103395,4.293338,15.570386,4.253840


In [7]:
# 连续3年收入占比及利润占比 扩大的行业
growth_ind=t[(t['profit_1_p']>t['profit_2_p']) & (t['profit_2_p']>t['profit_3_p']) &
     (t['revenue_1_p']>t['revenue_2_p']) & (t['revenue_2_p']>t['revenue_3_p'])]

growth_ind.sort_values(['weight'],ascending=False)

,weight,weight_money,market_cap,stock_cnt,revenue_1,profit_1,revenue_2,profit_2,revenue_3,profit_3,market_cap_percent,revenue_1_p,profit_1_p,revenue_2_p,profit_2_p,revenue_3_p,profit_3_p
sw_1,,,,,,,,,,,,,,,,,
食品饮料I,9.186,6.408974,25233.4475,9,3.279341e+11,8.617232e+10,2.783955e+11,6.818066e+10,2.250050e+11,4.751151e+10,6.903644,1.100094,2.965051,1.077004,2.522949,0.995317,2.046507
医药生物I,6.670,7.104281,17034.8707,30,5.939119e+11,4.835309e+10,4.794590e+11,4.387837e+10,4.016019e+11,3.597520e+10,4.660587,1.992347,1.663752,1.854841,1.623670,1.776500,1.549592
电子I,4.877,7.798681,13891.1435,19,9.325572e+11,5.307639e+10,4.455778e+11,4.166429e+10,3.592734e+11,2.934326e+10,3.800492,3.128373,1.826273,1.723768,1.541741,1.589258,1.263929
采掘I,1.669,1.844714,21062.6224,10,3.158201e+12,1.234404e+11,2.734897e+12,9.547352e+10,2.321330e+12,6.286078e+10,5.762544,10.594558,4.247385,10.580258,3.532891,10.268481,2.707659
商业贸易I,0.961,0.879024,3000.8262,5,3.688453e+11,2.161108e+10,2.894605e+11,1.088618e+10,2.261196e+11,5.473027e+09,0.820999,1.237335,0.743602,1.119811,0.402831,1.000248,0.235745
建筑材料I,0.932,0.994722,2735.9128,3,1.674578e+11,3.495779e+10,1.030206e+11,2.242326e+10,7.493483e+10,1.179954e+10,0.748521,0.561757,1.202841,0.398547,0.829748,0.331477,0.508252
休闲服务I,0.753,0.435041,1840.8269,2,5.517386e+10,5.575875e+09,3.441459e+10,4.117098e+09,2.568349e+10,2.782977e+09,0.503634,0.185087,0.191857,0.133137,0.152349,0.113612,0.119874


In [8]:
# 连续3年收入占比及利润占比 下滑的行业 
decline_ind=t[(t['profit_1_p']<t['profit_2_p']) & (t['profit_2_p']<t['profit_3_p']) &
     (t['revenue_1_p']<t['revenue_2_p']) & (t['revenue_2_p']<t['revenue_3_p'])]
decline_ind

,weight,weight_money,market_cap,stock_cnt,revenue_1,profit_1,revenue_2,profit_2,revenue_3,profit_3,market_cap_percent,revenue_1_p,profit_1_p,revenue_2_p,profit_2_p,revenue_3_p,profit_3_p
sw_1,,,,,,,,,,,,,,,,,
农林牧渔I,1.052,1.398675,3105.2687,4,1.502345e+11,5.258787e+09,1.350409e+11,7.383021e+09,1.197238e+11,6.980589e+09,0.849574,0.503979,0.180946,0.522421,0.273200,0.529602,0.300681
纺织服装I,0.133,0.066931,407.4960,1,1.939216e+10,3.533456e+09,1.882739e+10,3.449812e+09,1.702799e+10,3.175032e+09,0.111487,0.065053,0.121581,0.072836,0.127656,0.075324,0.136761
银行I,17.160,8.301136,103208.9876,21,4.343419e+12,1.493945e+12,3.886534e+12,1.409825e+12,3.709606e+12,1.323915e+12,28.237051,14.570513,51.404246,15.035496,52.169017,16.409566,57.026186


In [15]:

components_df_growth=components_df[[growth_ind.index.contains(i) for i in components_df['sw_1']]].sort_values(['market_cap'],ascending=False)

components_df_notgrowth=components_df[[not growth_ind.index.contains(i) for i in components_df['sw_1']]].sort_values(['market_cap'],ascending=False)

# 指数增强策略：
# 1）对应连续3年增长的行业，各个行业挑选前10个公司(指数权重行业内前10)，且需要其权重大于0.2
# 2）对应非连续3年增长的行业，各个行业挑选前5个公司，且需要其权重大于0.3

components_df_growth2=components_df_growth[(components_df_growth['sw1_weight_rank']<=8) & (components_df_growth['weight']>0.2)]

components_df_notgrowth2=components_df_notgrowth[(components_df_notgrowth['sw1_weight_rank']<=5) & (components_df_notgrowth['weight']>0.3)]

print('连续增长行业选股权重和:%.2f' % (components_df_growth2['weight'].sum()))
#print(components_df_growth2.describe())
print('非连续增长行行业选股权重和:%.2f' % (components_df_notgrowth2['weight'].sum()))
#print(components_df_notgrowth2.describe())

# 1）和2）合起来的投资组合，进行权重再平衡。
components_df_alpha=components_df_growth2['weight'].append(components_df_notgrowth2['weight'])
components_df_alpha=components_df_alpha/components_df_alpha.sum()*100

# 组合各股票的权重信息
components_df_alpha.sort_values(ascending=False).head(20)


连续增长行业选股权重和:18.88
非连续增长行行业选股权重和:41.47


code
601318.XSHG    12.649125
600519.XSHG     6.637725
600036.XSHG     4.818399
000651.XSHE     3.627055
601166.XSHG     3.366914
000333.XSHE     3.295665
000858.XSHE     2.694194
600887.XSHG     2.555011
600030.XSHG     2.458908
600276.XSHG     2.364462
601328.XSHG     2.332980
600016.XSHG     2.154030
601288.XSHG     1.953539
000002.XSHE     1.903831
601668.XSHG     1.744764
002415.XSHE     1.635406
601601.XSHG     1.542617
600837.XSHG     1.534332
600900.XSHG     1.509478
600048.XSHG     1.305673
Name: weight, dtype: float64

In [ ]:
# 使用 沪深300指数增强策略.py 创建一个策略并回测
# 看是否能跑赢基准

# 优化方向：进一步分析沪深300成分股，优化选股及优化权重分配。